In [ ]:
import pandas as pd

In [ ]:
books = pd.read_csv("../DATA/books_with_categories.csv")

books.head()

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification",model="j-hartmann/emotion-english-distilroberta-base",top_k=None,device="cpu")

classifier("I love you")

In [ ]:
books["description"][0]

In [ ]:
classifier(books["description"][0])

In [ ]:
classifier(books["description"][0].split("."))

In [ ]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [ ]:
sentences[0]

In [ ]:
predictions[0]

In [ ]:
sentences[3]

In [ ]:
sorted(predictions[0],key=lambda x: x["label"])

In [ ]:
import numpy as np

emotion_labels = ["anger","disgust","fear","joy","sadness","surprise","neutral"]

isbn = []
emotion_scores = {label:[] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
  per_emotion_scores = {label: [] for label in emotion_labels}
  for prediction in predictions:
    sorted_predictions = sorted(prediction,key=lambda x: x["label"])
    for index, label in enumerate(emotion_labels):
      per_emotion_scores[label].append(sorted_predictions[index]["score"])
      
  return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [ ]:
for i in range(10):
  isbn.append(books["isbn13"][i])
  sentences = books["description"][i].split(".")
  predictions = classifier(sentences)
  max_scores = calculate_max_emotion_scores(predictions)
  
  for label in emotion_labels:
    emotion_scores[label].append(max_scores[label])

In [ ]:
emotion_scores

In [ ]:

from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
  isbn.append(books["isbn13"][i])
  sentences = books["description"][i].split(".")
  predictions = classifier(sentences)
  max_scores = calculate_max_emotion_scores(predictions)
  for label in emotion_labels:
    emotion_scores[label].append(max_scores[label])

In [ ]:
emotions_df = pd.DataFrame(emotion_scores)

emotions_df["isbn13"] = isbn

In [ ]:
emotions_df.head()

In [ ]:
books = pd.merge(books,emotions_df,on="isbn13")

books.head()

In [ ]:
books.to_csv("../DATA/books_with_emotions.csv",index=False)